In [15]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from osgeo import gdal
from osgeo import osr

# HDF5 dosyasını yükleme
filename = r'C:\Users\Casper\Desktop\cumgeotif\cum.h5'
f = h5py.File(filename, 'r')

# Girdi verilerini al
imdates = f['imdates'][:]
data = f['cum'][:]

# Veri şeklini düzenleme
num_samples, height, width = data.shape
data = data.reshape(num_samples, height * width)
print("Veri şekli (num_samples, height, width):", data.shape)

# 'nan' değerleri 0 ile değiştirme
data = np.nan_to_num(data, nan=0.0)

# 'nan' değerleri komşu değerlerin ortalamasıyla doldurma
for i in range(num_samples):
    zero_indices = data[i] == 0
    non_zero_indices = np.logical_not(zero_indices)
    zero_count = np.sum(zero_indices)

    if zero_count > 0:
        # Komşu değerlerin ortalamasını hesaplama
        neighbor_values = np.zeros_like(data[i])
        for j in range(height * width):
            if zero_indices[j]:
                # Üst, alt, sol, sağ piksellerin değerlerini topla
                neighbors = []
                if j - width >= 0:
                    neighbors.append(data[i][j - width])  # üst piksel
                if j + width < height * width:
                    neighbors.append(data[i][j + width])  # alt piksel
                if j % width != 0:
                    neighbors.append(data[i][j - 1])  # sol piksel
                if (j + 1) % width != 0:
                    neighbors.append(data[i][j + 1])  # sağ piksel

                neighbor_values[j] = np.mean(neighbors)

        # 0 değerlerini komşu değerlerin ortalamasıyla doldurma
        data[i][zero_indices] = neighbor_values[zero_indices]


# Eğitim ve test veri setlerini oluşturma
train_size = 158  # İlk 158 veri örneği eğitim verisi olarak kullanılır
train_data = data[:train_size]
test_data = data[train_size:]
print("Eğitim veri seti şekli:", train_data.shape)
print("Test veri seti şekli:", test_data.shape)

# Girdi ve çıktı verilerini oluşturma
lookback = 8  # Tahmin için kullanılacak önceki gözlemler sayısı
X_train, y_train = [], []
for i in range(lookback, len(train_data)):
    X_train.append(train_data[i - lookback:i])
    y_train.append(train_data[i])
X_train, y_train = np.array(X_train), np.array(y_train)
print("Eğitim verisi X şekli:", X_train.shape)
print("Eğitim verisi y şekli:", y_train.shape)

X_test, y_test = [], []
for i in range(lookback, len(test_data)):
    X_test.append(test_data[i - lookback:i])
    y_test.append(test_data[i])
X_test, y_test = np.array(X_test), np.array(y_test)
print("Test verisi X şekli:", X_test.shape)
print("Test verisi y şekli:", y_test.shape)

# LSTM modelini oluşturma
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(lookback, height * width)))
model.add(LSTM(units=50))
model.add(Dense(units=height * width))
model.compile(optimizer='adam', loss='mean_squared_error')

# Modeli eğitme
print("Model eğitimi başlıyor...")
model.fit(X_train, y_train, epochs=100, batch_size=32)
print("Model eğitimi tamamlandı.")

# Modelin tahmin yapması
print("Tahmin yapılıyor...")
predicted = model.predict(X_test)
print("Tahminler yapıldı. Tahmin şekli:", predicted.shape)

# Tahmin sonuçlarını kaydetme
output_folder = r'C:\Users\Casper\Desktop\cumgeotif\predicted'
for i in range(len(predicted)):
    prediction = predicted[i].reshape((height, width))

    # Verileri GeoTIFF olarak kaydetme
    output_path = output_folder + f'/tahmin_{i+1}.tif'
    print("Kaydediliyor:", output_path)
    driver = gdal.GetDriverByName("GTiff")
    dataset = driver.Create(output_path, width, height, 1, gdal.GDT_UInt16)
    dataset.GetRasterBand(1).WriteArray(prediction)
    
    # Projeksiyon dönüşümü için ayarları yapma
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(3857)
    dataset.SetProjection(srs.ExportToWkt())

     # Veriye coğrafi dönüşüm bilgisini ekleme
    geotransform = (0, 1, 0, 0, 0, 1)  # Örnek bir coğrafi dönüşüm
    dataset.SetGeoTransform(geotransform)

    
    dataset.FlushCache()
    dataset = None

# HDF5 dosyasını kapatma
f.close()

Veri şekli (num_samples, height, width): (198, 29161)
Eğitim veri seti şekli: (158, 29161)
Test veri seti şekli: (40, 29161)
Eğitim verisi X şekli: (150, 8, 29161)
Eğitim verisi y şekli: (150, 29161)
Test verisi X şekli: (32, 8, 29161)
Test verisi y şekli: (32, 29161)
Model eğitimi başlıyor...
Epoch 1/100
5/5 [==============================] - 8s 361ms/step - loss: 177.9983
Epoch 2/100
5/5 [==============================] - 2s 312ms/step - loss: 176.8854
Epoch 3/100
5/5 [==============================] - 2s 328ms/step - loss: 174.9977
Epoch 4/100
5/5 [==============================] - 2s 329ms/step - loss: 172.3782
Epoch 5/100
5/5 [==============================] - 2s 331ms/step - loss: 169.2483
Epoch 6/100
5/5 [==============================] - 2s 318ms/step - loss: 165.8663
Epoch 7/100
5/5 [==============================] - 2s 317ms/step - loss: 162.4081
Epoch 8/100
5/5 [==============================] - 2s 317ms/step - loss: 159.0207
Epoch 9/100
5/5 [==============================] 

5/5 [==============================] - 2s 325ms/step - loss: 96.6398
Epoch 97/100
5/5 [==============================] - 2s 319ms/step - loss: 96.4676
Epoch 98/100
5/5 [==============================] - 2s 323ms/step - loss: 96.2917
Epoch 99/100
5/5 [==============================] - 2s 322ms/step - loss: 96.1226
Epoch 100/100
5/5 [==============================] - 2s 310ms/step - loss: 95.9385
Model eğitimi tamamlandı.
Tahmin yapılıyor...
1/1 [==============================] - 2s 2s/step
Tahminler yapıldı. Tahmin şekli: (32, 29161)
Kaydediliyor: C:\Users\Casper\Desktop\cumgeotif\predicted/tahmin_1.tif
Kaydediliyor: C:\Users\Casper\Desktop\cumgeotif\predicted/tahmin_2.tif
Kaydediliyor: C:\Users\Casper\Desktop\cumgeotif\predicted/tahmin_3.tif
Kaydediliyor: C:\Users\Casper\Desktop\cumgeotif\predicted/tahmin_4.tif
Kaydediliyor: C:\Users\Casper\Desktop\cumgeotif\predicted/tahmin_5.tif
Kaydediliyor: C:\Users\Casper\Desktop\cumgeotif\predicted/tahmin_6.tif
Kaydediliyor: C:\Users\Casper\Deskt